# Convolutional Neural Network

### Importing the libraries

In [20]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [21]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [22]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 5216 images belonging to 2 classes.


### Preprocessing the Test set

In [23]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 624 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [24]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [25]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [26]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [27]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [28]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [29]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [30]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [31]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [32]:
EPOCH_SIZE = 5 #change this to 25 for better accuracy, using 5 for speed
cnn.fit(x = training_set, validation_data = test_set, epochs = EPOCH_SIZE)

Epoch 1/5
163/163 [==============================] - 46s 276ms/step - loss: 0.3501 - accuracy: 0.8414 - val_loss: 0.3413 - val_accuracy: 0.8478
Epoch 2/5
163/163 [==============================] - 54s 334ms/step - loss: 0.2268 - accuracy: 0.9066 - val_loss: 0.4644 - val_accuracy: 0.7997
Epoch 3/5
163/163 [==============================] - 45s 279ms/step - loss: 0.2009 - accuracy: 0.9222 - val_loss: 0.3708 - val_accuracy: 0.8301
Epoch 4/5
163/163 [==============================] - 62s 380ms/step - loss: 0.1809 - accuracy: 0.9317 - val_loss: 0.2831 - val_accuracy: 0.8846
Epoch 5/5
163/163 [==============================] - 63s 386ms/step - loss: 0.1802 - accuracy: 0.9331 - val_loss: 0.3830 - val_accuracy: 0.8558


## Part 4 - Making a predictions

In [33]:
import os
import numpy as np
from keras.preprocessing import image
import random

PNEUMONIA_PREDICTION_THRESHOLD = 0.7 #Change it to 0.9 when using 25 epochs

def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(64, 64))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.  # Model was trained with inputs scaled to [0, 1]

    prediction = model.predict(img_tensor)
    return 'Pneumonia' if prediction[0][0] > PNEUMONIA_PREDICTION_THRESHOLD else 'Normal'

# Define your directories
normal_dir = 'val/NORMAL'
pneumonia_dir = 'val/PNEUMONIA'

# Get list of all files in each directory
normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir)]
pneumonia_files = [os.path.join(pneumonia_dir, f) for f in os.listdir(pneumonia_dir)]

# Combine the lists and shuffle
all_files = normal_files + pneumonia_files
random.shuffle(all_files)

# Predict each image in the shuffled list
for f in all_files:
    prediction = predict_image(cnn, f)
    print(f'Prediction for {os.path.basename(f)}: {prediction}\n')

1/1 [==============================] - 0s 142ms/step
Prediction for NORMAL2-IM-1430-0001.jpeg: Normal

1/1 [==============================] - 0s 25ms/step
Prediction for person1947_bacteria_4876.jpeg: Pneumonia

1/1 [==============================] - 0s 36ms/step
Prediction for person1951_bacteria_4882.jpeg: Pneumonia

1/1 [==============================] - 0s 43ms/step
Prediction for person1954_bacteria_4886.jpeg: Pneumonia

1/1 [==============================] - 0s 29ms/step
Prediction for NORMAL2-IM-1442-0001.jpeg: Normal

1/1 [==============================] - 0s 43ms/step
Prediction for person1946_bacteria_4875.jpeg: Pneumonia

1/1 [==============================] - 0s 24ms/step
Prediction for person1952_bacteria_4883.jpeg: Pneumonia

1/1 [==============================] - 0s 39ms/step
Prediction for NORMAL2-IM-1427-0001.jpeg: Pneumonia

1/1 [==============================] - 0s 28ms/step
Prediction for NORMAL2-IM-1431-0001.jpeg: Normal

1/1 [==============================] - 0s 3

In [34]:
print(prediction)

Pneumonia
